In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Forex predict using linear regression')\
        .config('hive.metastore.uris', 'thrift://10.121.31.83:9083')\
        .enableHiveSupport()\
        .getOrCreate()

In [5]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
|      person|
|       test2|
|     titanic|
|  titanic_db|
+------------+



In [2]:
spark.sql("use forex")
spark.sql("select * from 2012_m1_eurusd limit 10").show()
spark.sql("select count(distinct time) as count_time from 2012_m1_eurusd").show()
spark.sql("select count(distinct date) as count_time from 2012_m1_eurusd").show()
spark.sql("select count(distinct volume) as count_time from 2012_m1_eurusd").show()

AnalysisException: u"Database 'forex' not found;"

- Dữ liệu bắt đầu từ ngày 02-1-2012 đến ngày 31-12-2012
- Trong 1 ngày dữ liệu sẽ được cập nhật theo từng phút, bắt đầu từ 00.00 đến 23:59

In [25]:
spark.sql("SELECT MAX(time) FROM 2012_m1_eurusd").show()
spark.sql("SELECT MIN(time) FROM 2012_m1_eurusd").show()
spark.sql("SELECT MAX(date) FROM 2012_m1_eurusd").show()
spark.sql("SELECT MIN(date) FROM 2012_m1_eurusd").show()

+---------+
|max(time)|
+---------+
|    23:59|
+---------+

+---------+
|min(time)|
+---------+
|    00:00|
+---------+

+----------+
| max(date)|
+----------+
|2012.12.31|
+----------+

+----------+
| min(date)|
+----------+
|2012.01.02|
+----------+



In [38]:
spark.sql("SELECT MIN(high_bid) FROM 2012_m1_eurusd").show()
spark.sql("SELECT MIN(open_bid) FROM 2012_m1_eurusd").show()
spark.sql("SELECT MAX(high_bid) FROM 2012_m1_eurusd").show()
spark.sql("SELECT date,time,high_bid as min_high_bid FROM 2012_m1_eurusd WHERE high_bid=1.20474").show()
spark.sql("SELECT date,time,open_bid,high_bid as max_high_bid,low_bid,close_bid FROM 2012_m1_eurusd WHERE high_bid=1.34856").show()

+-------------+
|min(high_bid)|
+-------------+
|      1.20474|
+-------------+

+-------------+
|min(open_bid)|
+-------------+
|      1.20448|
+-------------+

+-------------+
|max(high_bid)|
+-------------+
|      1.34856|
+-------------+

+----------+-----+------------+
|      date| time|min_high_bid|
+----------+-----+------------+
|2012.07.24|12:32|     1.20474|
+----------+-----+------------+

+----------+-----+--------+------------+-------+---------+
|      date| time|open_bid|max_high_bid|low_bid|close_bid|
+----------+-----+--------+------------+-------+---------+
|2012.02.24|13:04| 1.34782|     1.34856|1.34782|  1.34806|
+----------+-----+--------+------------+-------+---------+



### Xóa column close_bid

In [21]:
df_forex = spark.sql("SELECT * FROM 2012_m1_eurusd")
df_forex = df_forex.drop('volume')
df_forex.show()

+----------+-----+--------+--------+-------+---------+
|      date| time|open_bid|high_bid|low_bid|close_bid|
+----------+-----+--------+--------+-------+---------+
|2012.01.02|02:00| 1.29324| 1.29381|1.29324|  1.29332|
|2012.01.02|02:01| 1.29326| 1.29345|1.29275|  1.29341|
|2012.01.02|02:02| 1.29342| 1.29344|1.29341|  1.29343|
|2012.01.02|02:03| 1.29342| 1.29342|1.29335|  1.29336|
|2012.01.02|02:04| 1.29324| 1.29347| 1.2932|  1.29346|
|2012.01.02|02:05| 1.29354| 1.29415|1.29354|  1.29392|
|2012.01.02|02:06| 1.29393| 1.29442|1.29391|   1.2943|
|2012.01.02|02:07| 1.29429| 1.29429|1.29395|  1.29395|
|2012.01.02|02:08| 1.29396| 1.29408|1.29395|  1.29407|
|2012.01.02|02:09| 1.29406| 1.29434|1.29401|   1.2942|
|2012.01.02|02:10| 1.29418| 1.29436|1.29408|  1.29425|
|2012.01.02|02:11| 1.29423| 1.29438|1.29423|  1.29433|
|2012.01.02|02:12| 1.29442| 1.29447|1.29401|  1.29432|
|2012.01.02|02:13| 1.29433| 1.29437|1.29421|  1.29422|
|2012.01.02|02:14| 1.29421| 1.29435|1.29403|  1.29405|
|2012.01.0